In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam

In [5]:
#read data
train_datagen=ImageDataGenerator(rescale=(1/255.)) 
IMG_SIZE = 64
#directory should be 'gap_images/gap_images/xxxx.jpg'
train_generator=train_datagen.flow_from_directory('gap_images/',#has to have subdirectory under it, otherwise 0 file found
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=64,
                                                 class_mode= 'input',
                                                 shuffle=True, 
                                                 classes= None)


Found 9294 images belonging to 1 classes.


In [6]:
h, w, d = train_generator[0][0].shape[1:]
input_layer = Input(shape=(h, w, d))
latent_dimension=4
encoded_layer1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
encoded_layer1 = MaxPooling2D( (2, 2), padding='same')(encoded_layer1)
encoded_layer2 = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded_layer1)
encoded_layer2 = MaxPooling2D( (2, 2), padding='same')(encoded_layer2)

encoded_layer3=Flatten()(encoded_layer2)
encoded_layer4=Dense(128, activation='relu', use_bias=True)(encoded_layer3)
latent_view =Dense(4, activation='linear', use_bias=True)(encoded_layer4)
decoded_layer1=Dense(128, activation='relu', use_bias=True)(latent_view)
decoded_layer2=Dense(8192, activation='relu', use_bias=True)(decoded_layer1)
decoded_layer3=Reshape(target_shape=(16,16,32))(decoded_layer2)

decoded_layer4 = Conv2D(32, (3, 3), activation='relu', padding='same')(decoded_layer3)
decoded_layer4 = UpSampling2D((2, 2))(decoded_layer4)
decoded_layer5 = Conv2D(64, (3, 3), activation='relu',padding='same')(decoded_layer4)
decoded_layer5 = UpSampling2D((2, 2))(decoded_layer5)
output_layer   = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(decoded_layer5)

cae = Model(input_layer, output_layer, name='ae_model')
cae.summary()

Model: "ae_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               104

In [7]:
#training: no decrease after 2 epochs
cae.compile(loss='mse', optimizer=Adam(lr=0.001))
step_size_train=train_generator.n//train_generator.batch_size
history = cae.fit_generator(generator=train_generator, 
                            steps_per_epoch=step_size_train,
                            #validation_data= train_generator,
                            epochs=2)

Epoch 1/2
145/145 [==============================] - 117s 807ms/step - loss: 0.0404
Epoch 2/2
145/145 [==============================] - 113s 780ms/step - loss: 0.0264


In [ ]:
#Save Model

model_json = cae.to_json()
with open("autoencoder.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cae.save_weights("ae_weights.h5")
print("Saved model to disk")